In [117]:
import numpy as np
from IPython.display import Image

# Definition of the gates
S = np.array([[1,0],
             [0, 1j]])

T = np.array([[1, 0],
             [0, np.exp(1j*np.pi/4)]])

H = (1/np.sqrt(2))*np.array([[1, 1],
              [1, -1]])

X = np.array([[0,1],[1,0]])
SW = np.array([[1, 0, 0, 0],
              [0,0,1,0],
              [0,1,0,0],
              [0,0,0,1]])

# Utility functions
def controlled_u(U_gate: np.array, control:bool ):
    """ 
    Generates the right U gate depending the number of qubits used in the problem.
    The ones don't go necessarily on top left, depends on who controls and who's controlled. 
    If control is off, upper qubit is the control one. Control up reverses the configuration.
    """
    
    dim = 2**2
    base = np.diag(*np.ones([1,4],dtype=complex))
    if control:
        base[0:2,:][:,0:2] = U_gate
    else:
        base[(dim-2):(dim),:][:,(dim-2):(dim)]  = U_gate
    return base

def unit(dim:int, pos: int):
    """ 
    Generates a unit vector with the one in the position pos
    """
    vec = np.zeros([dim,1])
    vec[pos] = 1
    return vec

def reorder_gate(G, perm):
    """
    Adapt gate 'G' to an ordering of the qubits as specified in 'perm'.
    Example, given G = np.kron(np.kron(A, B), C):
    reorder_gate(G, [1, 2, 0]) == np.kron(np.kron(B, C), A)
    """
    perm = list(perm)
    # number of qubits
    n = len(perm)
    # reorder both input and output dimensions
    perm2 = perm + [n + i for i in perm]
    return np.reshape(np.transpose(np.reshape(G, 2*n*[2]), perm2), (2**n, 2**n))

<img src="img/worksheet8.jpeg" width="600">



In [124]:
# We distinguish 7 moments (Concurrent sections of logic gates)
# We go from right to left
moments = []

# 7th slice
moments.append(reorder_gate(np.kron(SW, np.eye(2)), [0 ,2 ,1 ]))

# 6th slice
moments.append(np.kron(np.kron(np.eye(2), np.eye(2)), H))

# 5th slice
moments.append(np.kron(np.eye(2), controlled_u(S, 1)))

# 4th slice 
moments.append(np.kron(np.kron(np.eye(2), H),np.eye(2)))

# 3rd slice 
moments.append(reorder_gate(np.kron(controlled_u(T, 1), np.eye(2)), [0, 2, 1]))

# 2nd slice
moments.append(np.kron(controlled_u(S, 1), np.eye(2)))

# 1st slice
moments.append(np.kron(np.kron(H, np.eye(2)), np.eye(2)))


In [131]:
# To obtain the circuit matrix, we multiply all moments
circuit = moments[6]
for moment in reversed(moments[:6]):
    circuit = circuit @ moment
acircuit

array([[ 0.35355339+0.j        ,  0.35355339+0.j        ,
         0.35355339+0.j        ,  0.35355339+0.j        ,
         0.35355339+0.j        ,  0.35355339+0.j        ,
         0.35355339+0.j        ,  0.35355339+0.j        ],
       [-0.25      +0.25j      ,  0.        +0.35355339j,
         0.25      +0.25j      ,  0.35355339+0.j        ,
         0.25      -0.25j      ,  0.        -0.35355339j,
        -0.25      -0.25j      , -0.35355339+0.j        ],
       [ 0.        +0.35355339j,  0.35355339+0.j        ,
         0.        -0.35355339j, -0.35355339+0.j        ,
         0.        +0.35355339j,  0.35355339+0.j        ,
         0.        -0.35355339j, -0.35355339+0.j        ],
       [-0.25      -0.25j      ,  0.        +0.35355339j,
         0.25      -0.25j      , -0.35355339+0.j        ,
         0.25      +0.25j      ,  0.        -0.35355339j,
        -0.25      +0.25j      ,  0.35355339+0.j        ],
       [ 0.35355339+0.j        , -0.35355339+0.j        ,
         0

In [141]:
sample = np.array([[np.exp(2*np.pi*1j*j*k/8)/np.sqrt(8) for j in range(8)] for k in range(8)])


In [152]:
np.testing.assert_array_almost_equal(circuit, sample, 4, verbose=True)

AssertionError: 
Arrays are not almost equal to 4 decimals

Mismatch: 75%
Max absolute difference: 0.65328148
Max relative difference: 1.84775907
 x: array([[ 0.3536+0.j    ,  0.3536+0.j    ,  0.3536+0.j    ,
         0.3536+0.j    ,  0.3536+0.j    ,  0.3536+0.j    ,
         0.3536+0.j    ,  0.3536+0.j    ],...
 y: array([[ 3.5355e-01+0.0000e+00j,  3.5355e-01+0.0000e+00j,
         3.5355e-01+0.0000e+00j,  3.5355e-01+0.0000e+00j,
         3.5355e-01+0.0000e+00j,  3.5355e-01+0.0000e+00j,...

AssertionError: 
Not equal to tolerance rtol=0.1, atol=0

Mismatch: 75%
Max absolute difference: 0.65328148
Max relative difference: 1.84775907
 x: array([[ 0.353553+0.j      ,  0.353553+0.j      ,  0.353553+0.j      ,
         0.353553+0.j      ,  0.353553+0.j      ,  0.353553+0.j      ,
         0.353553+0.j      ,  0.353553+0.j      ],...
 y: array([[ 3.535534e-01+0.000000e+00j,  3.535534e-01+0.000000e+00j,
         3.535534e-01+0.000000e+00j,  3.535534e-01+0.000000e+00j,
         3.535534e-01+0.000000e+00j,  3.535534e-01+0.000000e+00j,...